
# 🔧 Merge Tool (Colab) — v2
- Устанавливает зависимости **напрямую из GitHub** (`requirements.txt` в вашем репозитории).
- Даёт интерфейс (Gradio) для **объединения двух файлов** (CSV/XLSX) → **Excel** на скачивание.
- **НОВОЕ:** сразу можно **скачать примеры входных файлов** (с первой ячейки и прямо в UI).


In [ ]:
#@title 📥 Создать и (при желании) скачать примерные файлы
import os, io
import pandas as pd

EX_DIR = "/content/examples"
os.makedirs(EX_DIR, exist_ok=True)

df1 = pd.DataFrame({
    "id": [1,2,3,4],
    "name": ["Alice","Bob","Carol","Dan"],
    "value1": [10,20,30,40]
})
p1 = os.path.join(EX_DIR, "example_first.csv")
df1.to_csv(p1, index=False, encoding="utf-8")

df2 = pd.DataFrame({
    "id": [3,4,5,6],
    "category": ["X","X","Y","Z"],
    "value2": [300,400,500,600]
})
p2 = os.path.join(EX_DIR, "example_second.xlsx")
df2.to_excel(p2, index=False)

print("Примеры созданы:")
print("-", p1)
print("-", p2)

# Для Colab — раскомментируйте для мгновенной загрузки в браузер:
# from google.colab import files
# files.download(p1)
# files.download(p2)


In [ ]:
#@title 🔧 Настройки GitHub (requirements из вашего репозитория)
ORG = "mctwork2"  #@param {type:"string"}
REPO = "tpok001"  #@param {type:"string"}
BRANCH = "main"  #@param {type:"string"}
REQ_PATH = "requirements.txt"  #@param {type:"string"}
RAW_REQ_URL = f"https://raw.githubusercontent.com/{ORG}/{REPO}/{BRANCH}/{REQ_PATH}"
print("Будет установлено из:", RAW_REQ_URL)

In [ ]:
#@title 📦 Установка зависимостей из GitHub (raw requirements.txt)
import subprocess, sys
print("Обновляем pip...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "pip"])
print("Ставим зависимости из:", RAW_REQ_URL)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", RAW_REQ_URL])
print("Готово!")

In [ ]:
#@title ▶️ Запуск приложения (Gradio) с примерами на скачивание
import os
import pandas as pd
import gradio as gr

try:
    import openpyxl  # noqa: F401
except Exception as e:
    raise RuntimeError("Не установлено 'openpyxl'. Добавьте 'openpyxl' в requirements.txt вашего репозитория.") from e

EX_DIR = "/content/examples"
os.makedirs(EX_DIR, exist_ok=True)
example_first = os.path.join(EX_DIR, "example_first.csv")
example_second = os.path.join(EX_DIR, "example_second.xlsx")

if not (os.path.exists(example_first) and os.path.exists(example_second)):
    df1 = pd.DataFrame({"id":[1,2,3,4],"name":["Alice","Bob","Carol","Dan"],"value1":[10,20,30,40]})
    df1.to_csv(example_first, index=False, encoding="utf-8")
    df2 = pd.DataFrame({"id":[3,4,5,6],"category":["X","X","Y","Z"],"value2":[300,400,500,600]})
    df2.to_excel(example_second, index=False)

def _read_table(file_obj):
    if file_obj is None:
        return None
    name = getattr(file_obj, "name", None) or ""
    ext = os.path.splitext(name)[1].lower()
    if ext in [".xlsx", ".xls"]:
        return pd.read_excel(file_obj.name)
    elif ext in [".csv", ".txt"]:
        for enc in ("utf-8-sig", "utf-8", "cp1251", "latin1"):
            try:
                return pd.read_csv(file_obj.name, encoding=enc)
            except Exception:
                continue
        return pd.read_csv(file_obj.name)
    else:
        try:
            return pd.read_excel(file_obj.name)
        except Exception:
            return pd.read_csv(file_obj.name)

def update_keys(file1, file2):
    df1 = _read_table(file1) if file1 else None
    df2 = _read_table(file2) if file2 else None
    cols1 = list(map(str, df1.columns)) if df1 is not None else []
    cols2 = list(map(str, df2.columns)) if df2 is not None else []
    default_left = default_right = None
    common = [c for c in cols1 if c in cols2]
    if common:
        default_left = common[0]
        default_right = common[0]
    elif cols1 and cols2:
        default_left = cols1[0]
        default_right = cols2[0]
    return gr.update(choices=cols1, value=default_left), gr.update(choices=cols2, value=default_right)

def merge_files(file1, file2, mode, left_key, right_key, how):
    df1 = _read_table(file1)
    df2 = _read_table(file2)
    if df1 is None or df2 is None:
        return None, None, "❗ Загрузите оба файла."
    if mode == "Склеить по столбцам (hconcat)":
        m = pd.concat([df1.reset_index(drop=True), df2.reset_index(drop=True)], axis=1)
    else:
        if left_key is None or right_key is None:
            return None, None, "❗ Выберите ключевые столбцы для объединения."
        if str(left_key) not in map(str, df1.columns) or str(right_key) not in map(str, df2.columns):
            return None, None, "❗ Указанные ключи не найдены в данных."
        m = pd.merge(df1, df2, how=how, left_on=str(left_key), right_on=str(right_key))
    out_path = "/content/merged_result.xlsx"
    m.to_excel(out_path, index=False)
    preview = m.iloc[:2000, :200]
    info = f"✅ Готово! Строк: {len(m):,}, Столбцов: {m.shape[1]:,}".replace(",", " ")
    return preview, out_path, info

with gr.Blocks(title="Merge Tool") as demo:
    gr.Markdown("## Объединение двух файлов (CSV/Excel) → Excel")
    with gr.Row():
        gr.Markdown("**📥 Примеры входных файлов:**")
        gr.File(value=example_first, label="Скачать example_first.csv", interactive=False)
        gr.File(value=example_second, label="Скачать example_second.xlsx", interactive=False)
    with gr.Row():
        file1 = gr.File(label="Первый файл (CSV/XLSX)", file_types=[".csv", ".xlsx", ".xls"], type="file")
        file2 = gr.File(label="Второй файл (CSV/XLSX)", file_types=[".csv", ".xlsx", ".xls"], type="file")
    with gr.Row():
        mode = gr.Radio(["Объединить по ключу (join)", "Склеить по столбцам (hconcat)"],
                        value="Объединить по ключу (join)", label="Способ объединения")
        how = gr.Dropdown(["inner", "left", "right", "outer"], value="inner", label="Тип join (если выбран join)")
    with gr.Row():
        left_key = gr.Dropdown(choices=[], label="Ключевой столбец в первом файле")
        right_key = gr.Dropdown(choices=[], label="Ключевой столбец во втором файле")
    merge_btn = gr.Button("🔄 Объединить")
    out_preview = gr.Dataframe(label="Предпросмотр результата (первые 2000 строк, 200 столбцов)")
    out_file = gr.File(label="Скачать результат (Excel)")
    out_info = gr.Markdown()
    file1.change(update_keys, [file1, file2], [left_key, right_key])
    file2.change(update_keys, [file1, file2], [left_key, right_key])
    merge_btn.click(merge_files, [file1, file2, mode, left_key, right_key, how], [out_preview, out_file, out_info])
demo.launch(share=True)